# Bulk grade students on canvas

In [1]:
import janitor
from canvasapi import Canvas
import requests
import re
import os
from pprint import pprint
from joblib import Parallel, delayed

## 0. Load assignment globals and filter valid and invalid submissions

In [2]:
API_URL = "https://canvas.ubc.ca/" # default is canvas.ubc
API_KEY = os.getenv("CANVAS_API")  # canvas.ubc instructor token
COURSE_CODE = 53659
canvas = Canvas(API_URL, API_KEY)
course = canvas.get_course(COURSE_CODE)

In [4]:
assignment_number = 664385
assignment = course.get_assignment(assignment_number)

rubric = (pd.DataFrame(assignment.rubric).set_index('id'))

all_submissions = list(assignment.get_submissions())
valid_submissions = [sub for sub in all_submissions if sub.submission_type is not None]
invalid_submission = [sub for sub in all_submissions if sub.submission_type is None]

autograde_rubric_id = rubric[rubric.description == "Autograded Exercises"].index.tolist()

In [14]:
[s.user_id for s in invalid_submission]

[354658, 465438, 484630, 519710, 521819, 587147]

### (Helpers)

In [9]:
def pmap(f, arr, n_jobs=-1, prefer='threads', verbose=10):
    return Parallel(n_jobs=n_jobs, prefer=prefer, verbose=verbose)(delayed(f)(i) for i in arr)

def get_grade_map_for_question(question_id: str):
    return (pd
     .DataFrame(rubric['ratings'].loc[question_id])
     .set_index('description')['points']
    ).to_dict()

def get_perfect(submissions: list):
    full_points = {r['id']:"A+" for r in assignment.rubric}
    return pd.DataFrame({s.user_id:full_points for s in submissions}).T

def autograde(submissions: list, autograde_rubric_id: str):    
    def _autograde(sub):
        try:
            with requests.session() as s:
                html = s.get(sub.attachments[0]['url']).text
                score = int(re.findall(r'Autograde points: (\d) / \d', html)[0])
        except:
            score = 0
        return (sub.user_id, score)
    
    auto_grade = pmap(_autograde, submissions, n_jobs=128) # runs in parallel
    df = pd.DataFrame(auto_grade).set_index(0)
    df.columns = [autograde_rubric_id]
    return df.astype(str)

def letters_to_points(scores: pd.DataFrame):
    def _letters_to_points(question: pd.Series):
        grade_map = get_grade_map_for_question(question.name)
        return question.replace(grade_map)

    return scores.apply(_letters_to_points)

def get_rubric_assessment(student_id: int, points: pd.DataFrame):
    return {k:{"points":str(v)} for k, v in points.loc[student_id].to_dict().items()}



## 3. Read csv, map letter grades to points

In [6]:
graded_scores = pd.read_csv("../marked/scores.csv", index_col=0, dtype=object)

In [7]:
graded_scores

,_4655,_7464,_3484,_8393,_522,_2766,_202,_9716,_3461,_195
1591,A,A+,A+,A+,A+,A+,A+,A+,A+,9
3077,A+,A+,A+,A+,0,0,A+,A+,A+,0
3262,A+,A+,A+,A+,A+,0,0,A+,A+,9
15450,A+,A+,A+,A+,0,0,A+,A+,A+,9
16724,A,A+,A+,A+,0,A+,A+,A+,A+,9
...,...,...,...,...,...,...,...,...,...,...
564304,A,A+,A+,A+,0,0,0,A+,A+,9
566247,A+,A+,A+,A+,A+,0,0,A+,A+,9
566269,A+,A+,A+,A+,A+,A+,A+,A+,A+,9
566490,A+,A+,A+,A+,A+,A+,A+,A+,A+,8


In [10]:
points = letters_to_points(graded_scores)

In [15]:
points

,_4655,_7464,_3484,_8393,_522,_2766,_202,_9716,_3461,_195
1591,3.48,0.95,0.95,0.95,0.95,0.95,0.95,2.85,2.85,9
3077,3.80,0.95,0.95,0.95,0,0,0.95,2.85,2.85,0
3262,3.80,0.95,0.95,0.95,0.95,0,0,2.85,2.85,9
15450,3.80,0.95,0.95,0.95,0,0,0.95,2.85,2.85,9
16724,3.48,0.95,0.95,0.95,0,0.95,0.95,2.85,2.85,9
...,...,...,...,...,...,...,...,...,...,...
564304,3.48,0.95,0.95,0.95,0,0,0,2.85,2.85,9
566247,3.80,0.95,0.95,0.95,0.95,0,0,2.85,2.85,9
566269,3.80,0.95,0.95,0.95,0.95,0.95,0.95,2.85,2.85,9
566490,3.80,0.95,0.95,0.95,0.95,0.95,0.95,2.85,2.85,8


## 4. Upload to canvas

In [16]:
for sub in valid_submissions:
    print(f"Uploading {sub.user_id}: ")
    print("---------------------------")
    rubric = get_rubric_assessment(sub.user_id, points)
    pprint(rubric)
    sub.edit(rubric_assessment=rubric)
    print("===========================")

Uploading 1591: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.48'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 3077: 
---------------------------
{'_195': {'points': '0'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 3262: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0'},
 '_2766': {'points': '0'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'

Uploading 219158: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 244866: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 268201: 
---------------------------
{'_195': {'points': '0'},
 '_202': {'points': '0'},
 '_2766': {'points': '0'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.48'},
 '_522': {'points': '0'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points

Uploading 462283: 
---------------------------
{'_195': {'points': '8'},
 '_202': {'points': '0'},
 '_2766': {'points': '0'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.48'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 462853: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.48'},
 '_522': {'points': '0'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 462910: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points':

Uploading 489950: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 490169: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0'},
 '_2766': {'points': '0'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 490794: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0'},
 '_2766': {'points': '0'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.48'},
 '_522': {'points': '0'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.

Uploading 518190: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 518354: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.48'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 518402: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'poi

Uploading 529125: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 529141: 
---------------------------
{'_195': {'points': '0'},
 '_202': {'points': '0'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.48'},
 '_522': {'points': '0'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'points': '2.85'}}
Uploading 529343: 
---------------------------
{'_195': {'points': '9'},
 '_202': {'points': '0.95'},
 '_2766': {'points': '0.95'},
 '_3461': {'points': '2.85'},
 '_3484': {'points': '0.95'},
 '_4655': {'points': '3.8'},
 '_522': {'points': '0.95'},
 '_7464': {'points': '0.95'},
 '_8393': {'points': '0.95'},
 '_9716': {'poi